In [1]:
import calendar_utils
import json

In [2]:
import importlib 
importlib.reload(calendar_utils)

<module 'calendar_utils' from 'C:\\Users\\ksjy\\Downloads\\conv_ai-main\\conv_ai-main\\calendar_utils.py'>

In [3]:
from dotenv import load_dotenv
import os
import re
import datetime

load_dotenv()

True

In [4]:
# Basic ChatGPT class
import openai
from openai import OpenAI


class BasicChatGPT:
  def __init__(self, openai_api_key, max_tokens=100):
    openai.api_key = openai_api_key
    self.max_tokens = max_tokens
    self.messages = []
    self.client = OpenAI()

  def call(self):
    response = self.client.chat.completions.create(
      model="gpt-4-1106-preview",
      # model="gpt-3.5-turbo",
      response_format={ "type": "json_object" },
      messages=self.messages,
      max_tokens=self.max_tokens
    )
    return response

  def prompt(self, text):
    # Update the messages so far
    self.messages.append({
        "role": "user",
        "content": text,
    })

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response["choices"][0]["message"]
    self.messages.append(message)

    return message["content"]



In [5]:
# Make a ChatGPT object
basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), max_tokens = 500)

In [6]:
# calendar service 정의하는 부분 -> credentials.json 이 경로에 있어야 하고 
# 그리고 팝업 뜨면 너가 authorize 해줘야함 그리고 너 노트북이나 데탑 로컬에서 해야 함
# https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list
service = calendar_utils.get_calendar_service()

In [17]:
def analysis_dialogue_gpt_call(user_text):
    input_text = f"""###Instruction: The assistant is an expert in analyzing conversations for schedule management. It takes the user's conversation as input and analyzes what tasks need to be done, by when, and how many detailed tasks the user desires. The assistant recognizes the user's conversation and performs accurate analysis.
Output Goals:
Target Task, Target Time, Maximum Number of Detailed Tasks

Considerations:

1. If the target task and target time are not clear in your analysis of the user's conversation, you must output a response stating, "Please suggest a target task or target time."
2. If the maximum number of detailed tasks is not clear in your analysis of the user's conversation, set that number to 3.

Please refer to the following examples for your response
Example 1:
Input: I need to complete a C++ programming assignment on matrix multiplication optimization within 3 days from today. Please divide it into 4 tasks for scheduling.
Output(JSON type):
Target Task: Completing a C++ programming assignment on matrix multiplication optimization,
Target Time: 3 days from today,
Maximum number of detailed tasks: four

Example 2:
Input: I need to select a paper on multi-task learning, familiarize myself with it, and then present it by next Monday.
Output(JSON type):
Target Task: Research and presentation on a multi-task learning paper,
Target Time: Next Monday,
Maximum number of detailed tasks: three

Example 3:
Input: I need to coordinate our team's business trip schedule with another team and submit a report on the travel plan by next Wednesday. Please suggest a schedule divided into 5 detailed tasks.
Output(JSON type):
Target Task: Coordinating our team's business trip schedule with another team and writing a report on the travel plan,
Target Time: Next Wednesday,
Maximum number of detailed tasks: five

###User Input:
"""
    input_text += user_text

    basic_chatgpt.messages.append({
        "role": "user",
        "content": input_text,
    })

    response = basic_chatgpt.call()
    message = response.choices[0].message.content
    init_result = json.loads(message)
    init_result = {key.lower(): value for key, value in init_result.items()}

    formatted_string = f"Target Task: {init_result_1['target task']},\nTarget Time: {init_result_1['target time']},\nMaximum number of detailed tasks: {init_result_1['maximum number of detailed tasks']}"

    print(formatted_string)

    return formatted_string

    

In [18]:
def create_schedule_dialogue_gpt_call(formatted_string):
    input_text = f"""###Instruction : Please assist in optimized schedule management. As a 'Schedule Management Application', you act to suggest necessary tasks for work input by users, manage time effectively, and aid in overall productivity enhancement. The 'Schedule Management Application' performs the following roles for the "Target Task" and "Target Time" input by the user:

Create the required subtasks for the 'Target Task'. Distribute the required subtasks for the 'Target Task' appropriately by 'Target Time'. Finally, the assistant outputs the distribution of detailed tasks by 'Target Time' in JSON format.

Restrictions: 
1. Between 00:00:00 and 09:00:00 in Asia/Seoul time is sleep time and is excluded from schedule distribution. 
2. Asia/Seoul time between 12:00:00 and 13:00:00 is lunch time and is excluded from the schedule distribution. 
3. The period between 18:00:00 and 20:00:00 in Asia/Seoul time is dinner time and is excluded from the schedule distribution.
4. Assistant only outputs JSON.
5. You must strictly follow the json format presented in the example.
6. Create as many detailed tasks as the suggested Maximum number of detailed tasks.

Considerations: 
1. You must consider the entire duration of the given schedule and distribute tasks so that they are not concentrated on specific days. Be sure to not concentrate your work on a specific day or time. 
2. The time allotted for a single detailed tasks must be no more than 3 hours.

Here is an example
User Input:
Today is November 21, 2023.
Target Task: I need to select and present a paper on deep learning.
Target Time: Next Monday.
Maximum number of detailed task : five

Assistant Output(Should be JSON format):
{{
"Tasks": [
{{
  "Task": "Selecting a Paper",
  "Start Time": "2023-11-21T09:00:00",
  "End Time": "2023-11-21T12:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Thoroughly Reading the Paper",
  "Start Time": "2023-11-21T13:00:00",
  "End Time": "2023-11-22T17:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Researching Background Information",
  "Start Time": "2023-11-23T09:00:00",
  "End Time": "2023-11-23T12:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Creating the Presentation",
  "Start Time": "2023-11-23T13:00:00",
  "End Time": "2023-11-24T17:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Rehearsing the Presentation",
  "Start Time": "2023-11-25T09:00:00",
  "End Time": "2023-11-26T17:00:00",
  "timeZone": 'Asia/Seoul'
}}
]
}}

###User Input:
"""
    
    current_date = datetime.datetime.now()
    formatted_date = current_date.strftime("Today is %B %d, %Y.")
    
    input_text = input_text + formatted_date + "\n" + formatted_string
    
    
    basic_chatgpt.messages.append({
        "role": "user",
        "content": input_text,
    })
    
    # Call ChatGPT
    response = basic_chatgpt.call()
    message = response.choices[0].message.content
    init_result = json.loads(message)
    
    print(message)

    return init_result

In [20]:
user_text = """
I need to complete a project on developing a chat system using ChatGPT. This task must be completed by next Wednesday. 
Please suggest a schedule divided into 4 detailed tasks.
"""

In [21]:
formatted_string = analysis_dialogue_gpt_call(user_text)

Target Task: Completing a project on developing a chat system using ChatGPT,
Target Time: Next Wednesday,
Maximum number of detailed tasks: four


In [22]:
formatted_string

'Target Task: Completing a project on developing a chat system using ChatGPT,\nTarget Time: Next Wednesday,\nMaximum number of detailed tasks: four'

In [23]:
schedule_result = create_schedule_dialogue_gpt_call(formatted_string)

{
"Tasks": [
{
  "Task": "Project Planning and Requirements Gathering",
  "Start Time": "2023-11-23T09:00:00",
  "End Time": "2023-11-23T12:00:00",
  "timeZone": "Asia/Seoul"
},
{
  "Task": "Setting Up ChatGPT Integration Environment",
  "Start Time": "2023-11-23T13:00:00",
  "End Time": "2023-11-23T16:00:00",
  "timeZone": "Asia/Seoul"
},
{
  "Task": "Coding Core Chat System Functionalities",
  "Start Time": "2023-11-24T09:00:00",
  "End Time": "2023-11-24T12:00:00",
  "timeZone": "Asia/Seoul"
},
{
  "Task": "Testing and Refining the Chat System",
  "Start Time": "2023-11-24T13:00:00",
  "End Time": "2023-11-24T16:00:00",
  "timeZone": "Asia/Seoul"
}
]
}


In [24]:
schedule_result

{'Tasks': [{'Task': 'Project Planning and Requirements Gathering',
   'Start Time': '2023-11-23T09:00:00',
   'End Time': '2023-11-23T12:00:00',
   'timeZone': 'Asia/Seoul'},
  {'Task': 'Setting Up ChatGPT Integration Environment',
   'Start Time': '2023-11-23T13:00:00',
   'End Time': '2023-11-23T16:00:00',
   'timeZone': 'Asia/Seoul'},
  {'Task': 'Coding Core Chat System Functionalities',
   'Start Time': '2023-11-24T09:00:00',
   'End Time': '2023-11-24T12:00:00',
   'timeZone': 'Asia/Seoul'},
  {'Task': 'Testing and Refining the Chat System',
   'Start Time': '2023-11-24T13:00:00',
   'End Time': '2023-11-24T16:00:00',
   'timeZone': 'Asia/Seoul'}]}

In [11]:
# first detect today and then find what date is today

input_text = f"""###Instruction: The assistant is an expert in analyzing conversations for schedule management. It takes the user's conversation as input and analyzes what tasks need to be done, by when, and how many detailed tasks the user desires. The assistant recognizes the user's conversation and performs accurate analysis.
Output Goals:
Target Task, Target Time, Maximum Number of Detailed Tasks

Considerations:

1. If the target task and target time are not clear in your analysis of the user's conversation, you must output a response stating, "Please suggest a target task or target time."
2. If the maximum number of detailed tasks is not clear in your analysis of the user's conversation, set that number to 3.

Please refer to the following examples for your response
Example 1:
Input: I need to complete a C++ programming assignment on matrix multiplication optimization within 3 days from today. Please divide it into 4 tasks for scheduling.
Output(JSON type):
Target Task: Completing a C++ programming assignment on matrix multiplication optimization,
Target Time: 3 days from today,
Maximum number of detailed tasks: four

Example 2:
Input: I need to select a paper on multi-task learning, familiarize myself with it, and then present it by next Monday.
Output(JSON type):
Target Task: Research and presentation on a multi-task learning paper,
Target Time: Next Monday,
Maximum number of detailed tasks: three

Example 3:
Input: I need to coordinate our team's business trip schedule with another team and submit a report on the travel plan by next Wednesday. Please suggest a schedule divided into 5 detailed tasks.
Output(JSON type):
Target Task: Coordinating our team's business trip schedule with another team and writing a report on the travel plan,
Target Time: Next Wednesday,
Maximum number of detailed tasks: five

###User Input:
"""

input_text += """
I need to complete a project on developing a chat system using ChatGPT. This task must be completed by next Wednesday. 
Please suggest a schedule divided into 4 detailed tasks.
"""


basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message_1 = response.choices[0].message.content
init_result_1 = json.loads(message_1)
init_result_1 = {key.lower(): value for key, value in init_result_1.items()}

print(message_1)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



{
  "Target Task": "Completing a project on developing a chat system using ChatGPT",
  "Target Time": "Next Wednesday",
  "Maximum number of detailed tasks": "four"
}


In [12]:
formatted_string = f"Target Task: {init_result_1['target task']},\nTarget Time: {init_result_1['target time']},\nMaximum number of detailed tasks: {init_result_1['maximum number of detailed tasks']}"

print(formatted_string)

Target Task: Completing a project on developing a chat system using ChatGPT,
Target Time: Next Wednesday,
Maximum number of detailed tasks: four


In [13]:
# first detect today and then find what date is today

input_text = f"""###Instruction : Please assist in optimized schedule management. As a 'Schedule Management Application', you act to suggest necessary tasks for work input by users, manage time effectively, and aid in overall productivity enhancement. The 'Schedule Management Application' performs the following roles for the "Target Task" and "Target Time" input by the user:

Create the required subtasks for the 'Target Task'. Distribute the required subtasks for the 'Target Task' appropriately by 'Target Time'. Finally, the assistant outputs the distribution of detailed tasks by 'Target Time' in JSON format.

Restrictions: 
1. Between 00:00:00 and 09:00:00 in Asia/Seoul time is sleep time and is excluded from schedule distribution. 
2. Asia/Seoul time between 12:00:00 and 13:00:00 is lunch time and is excluded from the schedule distribution. 
3. The period between 18:00:00 and 20:00:00 in Asia/Seoul time is dinner time and is excluded from the schedule distribution.
4. Assistant only outputs JSON.
5. You must strictly follow the json format presented in the example.
6. Create as many detailed tasks as the suggested Maximum number of detailed tasks.

Considerations: 
1. You must consider the entire duration of the given schedule and distribute tasks so that they are not concentrated on specific days. Be sure to not concentrate your work on a specific day or time. 
2. The time allotted for a single detailed tasks must be no more than 3 hours.

Here is an example
User Input:
Today is November 21, 2023.
Target Task: I need to select and present a paper on deep learning.
Target Time: Next Monday.
Maximum number of detailed task : five

Assistant Output(Should be JSON format):
{{
"Tasks": [
{{
  "Task": "Selecting a Paper",
  "Start Time": "2023-11-21T09:00:00",
  "End Time": "2023-11-21T12:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Thoroughly Reading the Paper",
  "Start Time": "2023-11-21T13:00:00",
  "End Time": "2023-11-22T17:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Researching Background Information",
  "Start Time": "2023-11-23T09:00:00",
  "End Time": "2023-11-23T12:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Creating the Presentation",
  "Start Time": "2023-11-23T13:00:00",
  "End Time": "2023-11-24T17:00:00",
  "timeZone": 'Asia/Seoul'
}},
{{
  "Task": "Rehearsing the Presentation",
  "Start Time": "2023-11-25T09:00:00",
  "End Time": "2023-11-26T17:00:00",
  "timeZone": 'Asia/Seoul'
}}
]
}}

###User Input:
"""

current_date = datetime.datetime.now()
formatted_date = current_date.strftime("Today is %B %d, %Y.")

input_text = input_text + formatted_date + "\n" + formatted_string


basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message_2 = response.choices[0].message.content
init_result_2 = json.loads(message_2)

print(message_2)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



{
  "Tasks": [
    {
      "Task": "Research and Documentation of Chat System Requirements",
      "Start Time": "2023-11-23T09:00:00",
      "End Time": "2023-11-23T12:00:00",
      "timeZone": "Asia/Seoul"
    },
    {
      "Task": "Designing the ChatGPT Integration Architecture",
      "Start Time": "2023-11-23T13:00:00",
      "End Time": "2023-11-23T16:00:00",
      "timeZone": "Asia/Seoul"
    },
    {
      "Task": "Implementing Chat Functionalities",
      "Start Time": "2023-11-24T09:00:00",
      "End Time": "2023-11-24T12:00:00",
      "timeZone": "Asia/Seoul"
    },
    {
      "Task": "Testing and Refining the Chat System",
      "Start Time": "2023-11-24T13:00:00",
      "End Time": "2023-11-24T16:00:00",
      "timeZone": "Asia/Seoul"
    }
  ]
}


In [14]:
def convert_to_new_format(old_data):
    if isinstance(old_data, str):
        old_data = json.loads(old_data)    
    new_data = []
    for item in old_data["Tasks"]:
        converted_item = {
            "summary": item["Task"],
            "start": {
                "dateTime": f"{item['Start Time']}+09:00",
                "timeZone": item["timeZone"]
            },
            "end": {
                "dateTime": f"{item['End Time']}+09:00",
                "timeZone": item["timeZone"]
            }
        }
        new_data.append(converted_item)
    return new_data

In [16]:
new_message = convert_to_new_format(init_result_2)
new_message

[{'summary': 'Research and Documentation of Chat System Requirements',
  'start': {'dateTime': '2023-11-23T09:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-11-23T12:00:00+09:00', 'timeZone': 'Asia/Seoul'}},
 {'summary': 'Designing the ChatGPT Integration Architecture',
  'start': {'dateTime': '2023-11-23T13:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-11-23T16:00:00+09:00', 'timeZone': 'Asia/Seoul'}},
 {'summary': 'Implementing Chat Functionalities',
  'start': {'dateTime': '2023-11-24T09:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-11-24T12:00:00+09:00', 'timeZone': 'Asia/Seoul'}},
 {'summary': 'Testing and Refining the Chat System',
  'start': {'dateTime': '2023-11-24T13:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-11-24T16:00:00+09:00', 'timeZone': 'Asia/Seoul'}}]

In [67]:
# mydate = datetime.datetime.strptime(init_result['date'], "%Y/%m/%d").isoformat() + "Z"
# mydate_p1 = (datetime.datetime.strptime(init_result['date'], "%Y/%m/%d") + datetime.timedelta(days=1)).isoformat() + "Z"
mydate = datetime.datetime.now(datetime.UTC).isoformat().split('+')[0] + "Z"
mydate_p1 = (datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=1)).isoformat().split('+')[0] + "Z"
print(mydate)
print(mydate_p1)

2023-11-21T12:27:31.941471Z
2023-11-22T12:27:31.941471Z


In [72]:
#https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list

calendar_utils.get_event_list_recent(timeMin=mydate, timeMax=mydate_p1)

2023-09-04T11:00:00+09:00 컴퓨팅 2
2023-09-04T14:00:00+09:00 MLDL2
2023-09-06T09:00:00+09:00 세미나
2023-11-15T20:00:00+09:00 대화형 자연어 처리 미팅
2023-11-22T12:30:00+09:00 수학 통계
2023-09-25T12:30:00+09:00 수학 통계
2023-11-22T17:00:00+09:00 기승 미팅


[{'kind': 'calendar#event',
  'etag': '"3387466391168000"',
  'id': '70r62or170q3cbb371hj0b9k6cq3cb9pcpi6cbb26cs38d316lgj4dhh70',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=NzByNjJvcjE3MHEzY2JiMzcxaGowYjlrNmNxM2NiOXBjcGk2Y2JiMjZjczM4ZDMxNmxnajRkaGg3MF8yMDIzMDkwNFQwMjAwMDBaIGtrc2twQHNudS5hYy5rcg',
  'created': '2023-09-03T09:26:35.000Z',
  'updated': '2023-09-03T09:26:35.584Z',
  'summary': '컴퓨팅 2',
  'creator': {'email': 'kkskp@snu.ac.kr', 'self': True},
  'organizer': {'email': 'kkskp@snu.ac.kr', 'self': True},
  'start': {'dateTime': '2023-09-04T11:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-09-04T12:15:00+09:00', 'timeZone': 'Asia/Seoul'},
  'recurrence': ['RRULE:FREQ=WEEKLY;WKST=SU;UNTIL=20231215T145959Z;INTERVAL=1;BYDAY=MO,WE'],
  'iCalUID': '70r62or170q3cbb371hj0b9k6cq3cb9pcpi6cbb26cs38d316lgj4dhh70@google.com',
  'sequence': 0,
  'reminders': {'useDefault': False,
   'overrides': [{'method': 'popup', 'minutes': 10}]

In [49]:
# 이거 내가 룰로 짜서 this monday -> 2023/11/27 이렇게 파싱하려고 했던 흔적임. 근데 그냥 chatgpt gpt4 한테 물어봤는데 잘해서 그냥 이거로 안했음..
# 그렇게 할 때의 input_text 는 이거였음 아래 참고

# input_text = """
# Detect any time-related word from the given input from the user.
# Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
# When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
# \n\n
# Example 1
# Input: I want to schedule a meeting at 5PM tomorrow.
# Output: tomorrow

# Example 2
# Input: What time does the class start next Friday?
# Output: next Friday

# Example 3
# Input: What is the date of this upcoming Friday?
# Output: this upcoming Friday
# \n\nInput: 
# """


# query_date = None
# found_time_related_phrase = 'this Monday'
# found_time_related_phrase = found_time_related_phrase.lower()
# if found_time_related_phrase == 'today':
#     query_date = datetime.datetime.today().date()
# elif found_time_related_phrase == 'tomorrow':
#     query_date = datetime.datetime.today().date() + datetime.timedelta(days=1)
# elif found_time_related_phrase == 'yesterday':
#     query_date = datetime.datetime.today().date() - datetime.timedelta(days=1)
# elif 'day' in found_time_related_phrase:
#     phrases = found_time_related_phrase.split()
#     prefix = phrases[0]
#     dow = phrases[-1]
#     dow2num={'monday':0, 'tuesday':1, 'wednesday':2, 'thursday':3, 'friday': 4, 'saturday':5, 'sunday':6}
#     today = datetime.date.today()
#     this_dow = today + datetime.timedelta( (dow2num[dow]-today.weekday()) % 7 )
#     if prefix == 'this': # dow closest from today in the future
#         query_date = this_dow
#     elif prefix == 'next': # dow one week from this dow
#         query_date = this_dow + datetime.timedelta(7)
#     elif prefix == 'last': # dow one week prior to this dow
#         query_date = this_dow - datetime.timedelta(7)
# print(query_date)

2023-11-27
